In [1]:
import random
import numpy as np
import time
import pandas as pd
from os.path import expanduser
import sys

In [2]:
import alphatims.bruker
import alphatims.utils

In [3]:
log_file_name = alphatims.utils.set_logger(
    log_file_name="tutorial_log.txt",
    overwrite=True
)

In [4]:
alphatims.utils.show_platform_info()

2021-08-13 09:50:38> Platform information:
2021-08-13 09:50:38> system        - Linux
2021-08-13 09:50:38> release       - 4.15.0-153-generic
2021-08-13 09:50:38> version       - #160-Ubuntu SMP Thu Jul 29 06:54:29 UTC 2021
2021-08-13 09:50:38> machine       - x86_64
2021-08-13 09:50:38> processor     - x86_64
2021-08-13 09:50:38> cpu count     - 12
2021-08-13 09:50:38> cpu frequency - 1336.71 Mhz
2021-08-13 09:50:38> ram           - 61.6/62.8 Gb (available/total)
2021-08-13 09:50:38> 


In [5]:
alphatims.utils.show_python_info()

2021-08-13 09:50:38> Python information:
2021-08-13 09:50:38> alphatims  - 0.2.8
2021-08-13 09:50:38> bokeh      - 
2021-08-13 09:50:38> click      - 8.0.1
2021-08-13 09:50:38> datashader - 
2021-08-13 09:50:38> h5py       - 3.1.0
2021-08-13 09:50:38> hvplot     - 
2021-08-13 09:50:38> numba      - 0.53.1
2021-08-13 09:50:38> pandas     - 1.3.1
2021-08-13 09:50:38> psutil     - 5.8.0
2021-08-13 09:50:38> python     - 3.8.5
2021-08-13 09:50:38> python-lzf - 
2021-08-13 09:50:38> pyzstd     - 0.14.4
2021-08-13 09:50:38> selenium   - 
2021-08-13 09:50:38> tqdm       - 4.61.1
2021-08-13 09:50:38> 


In [6]:
file_name = '/media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d'

In [31]:
rt_lower = 1650.
rt_upper = 2200.
mz_lower = 780.
mz_upper = 795.
scan_lower = 450
scan_upper = 900

In [9]:
data = alphatims.bruker.TimsTOF(file_name)

2021-08-13 09:51:01> Importing data from /media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d
2021-08-13 09:51:01> Reading frame metadata for /media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d
2021-08-13 09:51:02> Reading 25,701 frames with 1,292,274,220 detector strikes for /media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d


100%|██████████| 25701/25701 [00:22<00:00, 1163.38it/s]


2021-08-13 09:51:24> Indexing /media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d...
2021-08-13 09:51:24> Opening handle for /media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d
2021-08-13 09:51:24> Fetching mobility values from /media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d
2021-08-13 09:51:24> Closing handle for /media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d
2021-08-13 09:51:24> Opening handle for /media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d
2021-08-13 09:51:24> Fetching mz values from /media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d
2021-08-13 09:51:24> Closing handle for /media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d
2021-08-13 09:51:26> Succesfully imported data from /media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d


In [40]:
df = data[
    {
        "rt_values": slice(rt_lower, rt_upper),
        "mz_values": slice(mz_lower, mz_upper),
        "scan_indices": slice(scan_lower, scan_upper),
        "precursor_indices": 0
    }
]

In [41]:
df.columns

Index(['raw_indices', 'frame_indices', 'scan_indices', 'precursor_indices',
       'push_indices', 'tof_indices', 'rt_values', 'rt_values_min',
       'mobility_values', 'quad_low_mz_values', 'quad_high_mz_values',
       'mz_values', 'intensity_values'],
      dtype='object')

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4564742 entries, 0 to 4564741
Data columns (total 13 columns):
 #   Column               Dtype  
---  ------               -----  
 0   raw_indices          int64  
 1   frame_indices        int64  
 2   scan_indices         int64  
 3   precursor_indices    int64  
 4   push_indices         int64  
 5   tof_indices          uint32 
 6   rt_values            float64
 7   rt_values_min        float64
 8   mobility_values      float64
 9   quad_low_mz_values   float64
 10  quad_high_mz_values  float64
 11  mz_values            float64
 12  intensity_values     uint16 
dtypes: float64(6), int64(5), uint16(1), uint32(1)
memory usage: 409.2 MB


In [43]:
df.rt_values.max()

2199.8037915865807

#### save the raw data as an HDF

In [13]:
hdf_file_name = data.save_as_hdf(
    directory=expanduser('~'),
    file_name=f"{data.sample_name}.hdf",
    overwrite=True
)

2021-08-13 09:51:44> Writing TimsTOF data to /home/daryl/P3856_YHE211_1_Slot1-1_1_5104.hdf.


100%|██████████| 31/31 [00:04<00:00,  7.51it/s]


2021-08-13 09:52:02> Succesfully wrote TimsTOF data to /home/daryl/P3856_YHE211_1_Slot1-1_1_5104.hdf.


In [3]:
hdf_file_name = '/home/daryl/P3856_YHE211_1_Slot1-1_1_5104.hdf'

In [4]:
data = alphatims.bruker.TimsTOF(hdf_file_name)

#### loading ms2

In [60]:
df = data[
    {
        "frame_indices": slice(int(14700), int(14900)),
        "scan_indices": slice(scan_lower, scan_upper),
        "precursor_indices": slice(1, None)
    }
]

In [61]:
df

,raw_indices,frame_indices,scan_indices,precursor_indices,push_indices,tof_indices,rt_values,rt_values_min,mobility_values,quad_low_mz_values,quad_high_mz_values,mz_values,intensity_values
0,790180764,14700,450,8981,13509750,151069,1659.761187,27.662686,1.089501,855.870619,858.870619,466.238998,52
1,790180765,14700,450,8981,13509750,170186,1659.761187,27.662686,1.089501,855.870619,858.870619,533.222076,83
2,790180766,14700,450,8981,13509750,180778,1659.761187,27.662686,1.089501,855.870619,858.870619,572.269738,10
3,790180767,14700,450,8981,13509750,194875,1659.761187,27.662686,1.089501,855.870619,858.870619,626.378707,40
4,790180768,14700,450,8981,13509750,241416,1659.761187,27.662686,1.089501,855.870619,858.870619,822.371458,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512550,800163293,14899,877,10457,13693058,209313,1680.823740,28.013729,0.871247,539.584443,541.584443,684.329791,44
512551,800163294,14899,877,10457,13693058,231654,1680.823740,28.013729,0.871247,539.584443,541.584443,779.054216,9
512552,800163295,14899,877,10457,13693058,299518,1680.823740,28.013729,0.871247,539.584443,541.584443,1104.434468,9
512553,800163296,14899,878,10457,13693059,371418,1680.823740,28.013729,0.870733,539.584443,541.584443,1510.955775,9
